#### Instructions 

#### Prioritize the MVP

In the previous lab, you had to scrape data about "hot songs". It's critical to be on track with that part, as it was part of the request from the CTO.
 - Done

If you couldn't finish the first lab, use this time to go back there.

**User experience:**

- What happens if the user inputs a song that doesn't exist?
- We return an error message
- What do we do with songs that have the same name, but a different artist?
- We can ask the user to type the artist name
- How do we deal with typos?

**Architecture:**

- Do we build the interaction with the user in the same notebook as the web-scraping?
- Where do we store the scraped songs?

**Scheduling / Automation:**

- Should we scrape billboard / wikipedia every time a user sends a request?

**Testing:**

- Does it work when you test it with a real user (a colleague)?

Chances are that more issues will appear, and that not all of them will be solved during this session. But what's important is that the issues have been identified.

In [1]:
def billboard_scraping():
    # 1. import libraries
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd

    # 2. find url and store it in a variable
    url = "https://www.billboard.com/charts/hot-100/"

    # 3. download html with a get request 
    response = requests.get(url)

    # 4.1. parse html (create the 'soup')
    soup = BeautifulSoup(response.content, "html.parser")

    s = soup.find_all('h3', class_='c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only')

    t = soup.find_all('h3', class_='c-title a-font-primary-bold-l a-font-primary-bold-m@mobile-max lrv-u-color-black u-color-white@mobile-max lrv-u-margin-r-150')
    
    #5 iterate over list
    title = []
    title.append(t[0].get_text().replace("\n",""))
    for i in range(len(s)):
        text = str(s[i].get_text()).replace("\n", "")
        title.append(text)

    s = soup.find_all('span', class_='c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only')

    t = soup.find_all('p', class_='c-tagline a-font-primary-l a-font-primary-m@mobile-max lrv-u-color-black u-color-white@mobile-max lrv-u-margin-tb-00 lrv-u-padding-t-025 lrv-u-margin-r-150')

    artist = []
    artist.append(t[0].get_text().replace("\n",""))
    for i in range(len(s)):
        text = str(s[i].get_text()).replace("\n", "")
        artist.append(text)
    
    #6 create the dataframe
    global billboard
    billboard = pd.DataFrame({"title":title,
                           "artist":artist,
                          })

    return

billboard_scraping()

In [2]:
def user_input():
    global x
    x = input("Please enter the title of a song: ")
    
    return

In [76]:
def lowercase_values(dataframe):
    for column_name in list(dataframe.select_dtypes(include='object').columns.values):
        dataframe[column_name] = dataframe[column_name].str.lower()


In [77]:
#The goal here is to print the title and the artist if the researched song is in the billboard hot 100
#if method
# issue : the result is printed in a table form and I want only the text, this method requires exact title and can be improved

user_input()
lowercase_values(billboard)
my_list = list(billboard["title"])
if str.lower(x) in my_list:
    print(billboard[billboard["title"] == str.lower(x)])
    print(billboard.sample(n = 1))
else:
    print("The song is not in the hot list")   

Please enter the title of a song: Knife Talk
         title                                   artist
26  knife talk  drake featuring 21 savage & project pat
            title                   artist
15  industry baby  lil nas x & jack harlow


# Lab | API wrappers - Create your collection of songs & audio features


#### Instructions 


To move forward with the project, you need to create a collection of songs with their audio features - as large as possible! 

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster.
The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!


Billboard Hot 100 playlist

https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ?si=19d7543e2fb94fe6

Chill Pill playlist

https://open.spotify.com/playlist/7aZ5mBWDDCNy7wmdCf4FCX?si=d28cf9ef49bd497a

https://www.geeksforgeeks.org/append-list-of-dictionary-and-series-to-a-existing-pandas-dataframe-in-python/

In [79]:
import config

In [82]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

results = sp.search(q="Lose Yourself",limit=1,market="GB")
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Lose+Yourself&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'},
       'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR',
       'id': '7dGJo4pcD2V6oG8kP0tJRR',
       'name': 'Eminem',
       'type': 'artist',
       'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/1rfORa9iYmocEsnnZGMVC4'},
     'href': 'https://api.spotify.com/v1/albums/1rfORa9iYmocEsnnZGMVC4',
     'id': '1rfORa9iYmocEsnnZGMVC4',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273b6ef2ebd34efb08cb76f6eec',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02b6ef2ebd34efb08cb76f6eec',
       'width': 300},
      {'height': 64,
       'url': 'https://i.

In [84]:
chillpill = sp.user_playlist_tracks("spotify", "7aZ5mBWDDCNy7wmdCf4FCX",market="GB")

In [86]:
import pprint

pprint.pprint(chillpill)


{'href': 'https://api.spotify.com/v1/playlists/7aZ5mBWDDCNy7wmdCf4FCX/tracks?offset=0&limit=100&market=GB&additional_types=track',
 'items': [{'added_at': '2020-01-14T14:23:30Z',
            'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/solenita'},
                         'href': 'https://api.spotify.com/v1/users/solenita',
                         'id': 'solenita',
                         'type': 'user',
                         'uri': 'spotify:user:solenita'},
            'is_local': False,
            'primary_color': None,
            'track': {'album': {'album_type': 'single',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
                                             'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
                                             'id': '0LyfQWJT6nXafLPZqxe9Of',
                                             'name': 'Vari

            'track': {'album': {'album_type': 'album',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6rqlONGmPuP2wJVSfliLBI'},
                                             'href': 'https://api.spotify.com/v1/artists/6rqlONGmPuP2wJVSfliLBI',
                                             'id': '6rqlONGmPuP2wJVSfliLBI',
                                             'name': 'Peter Fox',
                                             'type': 'artist',
                                             'uri': 'spotify:artist:6rqlONGmPuP2wJVSfliLBI'}],
                                'external_urls': {'spotify': 'https://open.spotify.com/album/6cEVfMd0XVocPbRrYkVY5H'},
                                'href': 'https://api.spotify.com/v1/albums/6cEVfMd0XVocPbRrYkVY5H',
                                'id': '6cEVfMd0XVocPbRrYkVY5H',
                                'images': [{'height': 640,
                                            'url': 'htt

                                   'type': 'artist',
                                   'uri': 'spotify:artist:6sz0k1q2aEtG5dxEgr4YWV'}],
                      'disc_number': 1,
                      'duration_ms': 386373,
                      'episode': False,
                      'explicit': False,
                      'external_ids': {'isrc': 'FRYAT1300040'},
                      'external_urls': {'spotify': 'https://open.spotify.com/track/3rCJPCJTJRUxvCduWDVprx'},
                      'href': 'https://api.spotify.com/v1/tracks/3rCJPCJTJRUxvCduWDVprx',
                      'id': '3rCJPCJTJRUxvCduWDVprx',
                      'is_local': False,
                      'is_playable': True,
                      'name': 'Ka moun kè',
                      'popularity': 48,
                      'preview_url': 'https://p.scdn.co/mp3-preview/29c4bf7649cfa8161dd15d4155c4d986ff129e00?cid=2b56d6e548bf4f0ca8a909428b1a3ea4',
                      'track': True,
                      'tra

                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/01C9OoXDvCKkGcf735Tcfo'},
                                             'href': 'https://api.spotify.com/v1/artists/01C9OoXDvCKkGcf735Tcfo',
                                             'id': '01C9OoXDvCKkGcf735Tcfo',
                                             'name': 'Serge Gainsbourg',
                                             'type': 'artist',
                                             'uri': 'spotify:artist:01C9OoXDvCKkGcf735Tcfo'}],
                                'external_urls': {'spotify': 'https://open.spotify.com/album/437n44rXu1XPP7pR075Fek'},
                                'href': 'https://api.spotify.com/v1/albums/437n44rXu1XPP7pR075Fek',
                                'id': '437n44rXu1XPP7pR075Fek',
                                'images': [{'height': 640,
                                            'url': 'https://i.scdn.co/image/ab67616d0000b27371f689e3278

                      'duration_ms': 214946,
                      'episode': False,
                      'explicit': False,
                      'external_ids': {'isrc': 'GBUM70603730'},
                      'external_urls': {'spotify': 'https://open.spotify.com/track/3N4DI1vuTSX1tz7fa2NQZw'},
                      'href': 'https://api.spotify.com/v1/tracks/3N4DI1vuTSX1tz7fa2NQZw',
                      'id': '3N4DI1vuTSX1tz7fa2NQZw',
                      'is_local': False,
                      'is_playable': True,
                      'linked_from': {'external_urls': {'spotify': 'https://open.spotify.com/track/4osg3vT6sXv6wNxm9Z6ucQ'},
                                      'href': 'https://api.spotify.com/v1/tracks/4osg3vT6sXv6wNxm9Z6ucQ',
                                      'id': '4osg3vT6sXv6wNxm9Z6ucQ',
                                      'type': 'track',
                                      'uri': 'spotify:track:4osg3vT6sXv6wNxm9Z6ucQ'},
                      'name': '

...

                      'duration_ms': 276720,
                      'episode': False,
                      'explicit': False,
                      'external_ids': {'isrc': 'GBUM71028665'},
                      'external_urls': {'spotify': 'https://open.spotify.com/track/33BnSMHgX0AsbKSIbkuMwh'},
                      'href': 'https://api.spotify.com/v1/tracks/33BnSMHgX0AsbKSIbkuMwh',
                      'id': '33BnSMHgX0AsbKSIbkuMwh',
                      'is_local': False,
                      'is_playable': True,
                      'linked_from': {'external_urls': {'spotify': 'https://open.spotify.com/track/2S3dAq95AG0vxAYIU8ajNI'},
                                      'href': 'https://api.spotify.com/v1/tracks/2S3dAq95AG0vxAYIU8ajNI',
                                      'id': '2S3dAq95AG0vxAYIU8ajNI',
                                      'type': 'track',
                                      'uri': 'spotify:track:2S3dAq95AG0vxAYIU8ajNI'},
                      'name': '

           {'added_at': '2015-09-12T14:33:45Z',
            'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/solenita'},
                         'href': 'https://api.spotify.com/v1/users/solenita',
                         'id': 'solenita',
                         'type': 'user',
                         'uri': 'spotify:user:solenita'},
            'is_local': False,
            'primary_color': None,
            'track': {'album': {'album_type': 'single',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7rtM2wPKQlFpsm0C4qJlDk'},
                                             'href': 'https://api.spotify.com/v1/artists/7rtM2wPKQlFpsm0C4qJlDk',
                                             'id': '7rtM2wPKQlFpsm0C4qJlDk',
                                             'name': 'Kendra Morris',
                                             'type': 'artist',
                                             'uri': 'spo

                                           {'height': 300,
                                            'url': 'https://i.scdn.co/image/ab67616d00001e024d45aa437d7669555e5ef835',
                                            'width': 300},
                                           {'height': 64,
                                            'url': 'https://i.scdn.co/image/ab67616d000048514d45aa437d7669555e5ef835',
                                            'width': 64}],
                                'name': 'Sobre Noites e Dias',
                                'release_date': '2014-10-13',
                                'release_date_precision': 'day',
                                'total_tracks': 10,
                                'type': 'album',
                                'uri': 'spotify:album:6IbOeTu897HkvhY6tpE5EY'},
                      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/67muZL7DIwZwLnH9BFfI5Z'},
                                 

            'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/solenita'},
                         'href': 'https://api.spotify.com/v1/users/solenita',
                         'id': 'solenita',
                         'type': 'user',
                         'uri': 'spotify:user:solenita'},
            'is_local': False,
            'primary_color': None,
            'track': {'album': {'album_type': 'single',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/41Ue54Vb6iWx2dcdRCM6oH'},
                                             'href': 'https://api.spotify.com/v1/artists/41Ue54Vb6iWx2dcdRCM6oH',
                                             'id': '41Ue54Vb6iWx2dcdRCM6oH',
                                             'name': 'Kazy Lambist',
                                             'type': 'artist',
                                             'uri': 'spotify:artist:41Ue54Vb6iWx2dcdRCM6oH'}],
          

                                'id': '2eOD1woggAOyLtM0UqORrk',
                                'images': [{'height': 640,
                                            'url': 'https://i.scdn.co/image/ab67616d0000b2734194177ed1419ab5d1e0b119',
                                            'width': 640},
                                           {'height': 300,
                                            'url': 'https://i.scdn.co/image/ab67616d00001e024194177ed1419ab5d1e0b119',
                                            'width': 300},
                                           {'height': 64,
                                            'url': 'https://i.scdn.co/image/ab67616d000048514194177ed1419ab5d1e0b119',
                                            'width': 64}],
                                'name': "L'Intégrale Des Enregistrements "
                                        'Orlando',
                                'release_date': '1999-01-01',
                                'relea

In [102]:
chillpill["items"][1]["track"]["external_urls"]["spotify"][31:]

'3GmYxpXRc62mj4SuJ4WEsl'

In [114]:
sp.audio_features(chillpill["items"][2]["track"]["external_urls"]["spotify"][31:])[0]

{'danceability': 0.749,
 'energy': 0.477,
 'key': 4,
 'loudness': -6.06,
 'mode': 0,
 'speechiness': 0.198,
 'acousticness': 0.153,
 'instrumentalness': 1.08e-05,
 'liveness': 0.823,
 'valence': 0.882,
 'tempo': 95.968,
 'type': 'audio_features',
 'id': '1pur8kv69XmZqA3uoMEYnH',
 'uri': 'spotify:track:1pur8kv69XmZqA3uoMEYnH',
 'track_href': 'https://api.spotify.com/v1/tracks/1pur8kv69XmZqA3uoMEYnH',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1pur8kv69XmZqA3uoMEYnH',
 'duration_ms': 133893,
 'time_signature': 4}

In [115]:
import pandas as pd
df = pd.DataFrame({})
df

""


In [118]:
for i in range(len(chillpill["items"])):
    audio_feature_dict = sp.audio_features(chillpill["items"][i]["track"]["external_urls"]["spotify"][31:])[0]
    my_dict_new = { key: [audio_feature_dict[key]] for key in list(audio_feature_dict.keys()) }
    df = df.append(my_dict_new, ignore_index=True, sort=False)

IndentationError: unexpected indent (2165954272.py, line 3)

In [ ]:
df